In [1]:
import nltk
import numpy as np
import pandas as pd
import math
import string
from collections import defaultdict
from nltk.corpus import floresta

# Завантаження даних NLTK
nltk.download('floresta')
nltk.download('punkt')

# --- 1. МОРФОЛОГІЧНІ ПРАВИЛА ДЛЯ ПОРТУГАЛЬСЬКОЇ МОВИ ---

punct = set(string.punctuation + "«»„“—…")

def assign_unk_pt(tok):
    """
    Призначення міток для невідомих слів (Португальська мова)
    """
    # Цифри
    if any(char.isdigit() for char in tok):
        return "--unk_digit--"

    # Пунктуація
    elif any(char in punct for char in tok):
        return "--unk_punct--"

    # Великі літери
    elif any(char.isupper() for char in tok):
        return "--unk_upper--"

    # Іменники (substantivos)
    # -ção (ação), -dade (liberdade), -mento (momento), -ismo, -ista
    elif any(tok.endswith(suffix) for suffix in ["cao", "ção", "dade", "mento", "ismo", "ista", "or", "s"]):
        return "--unk_noun--"

    # Дієслова (verbos)
    # -ar, -er, -ir, -ando, -endo, -indo, -u, -a, -e
    elif any(tok.endswith(suffix) for suffix in ["ar", "er", "ir", "ando", "endo", "indo", "am", "em"]):
        return "--unk_verb--"

    # Прикметники (adjetivos)
    # -oso, -vel, -al, -ico
    elif any(tok.endswith(suffix) for suffix in ["oso", "osa", "vel", "al", "ico", "ica"]):
        return "--unk_adj--"

    # Прислівники (advérbios)
    # -mente
    elif any(tok.endswith(suffix) for suffix in ["mente"]):
        return "--unk_adv--"

    return "--unk--"

# --- 2. ПІДГОТОВКА ДАНИХ (FLORESTA) ---

def get_floresta_data():
    """
    Завантажує корпус Floresta, спрощує теги та розділяє на train/test.
    """
    # Завантажуємо речення. Floresta має теги типу 'n+art', беремо лише 'n' (першу частину)
    # або останню частину, залежно від стратегії. Тут візьмемо останню частину (спрощено).
    tagged_sents = []
    for sent in floresta.tagged_sents():
        simplified_sent = []
        for word, tag in sent:
            # Очищення тегу (видалення метаданих)
            # Наприклад: "H+n" -> "n", "art" -> "art"
            if '+' in tag:
                clean_tag = tag.split('+')[-1] # Беремо частину після +
            else:
                clean_tag = tag
            simplified_sent.append((word.lower(), clean_tag)) # Lowercase для слів
        tagged_sents.append(simplified_sent)

    # Розділення 80/20
    split = int(len(tagged_sents) * 0.8)
    train_data = tagged_sents[:split]
    test_data = tagged_sents[split:]

    return train_data, test_data

# Отримуємо дані
train_sents, test_sents = get_floresta_data()
print(f"Кількість речень у навчальній вибірці: {len(train_sents)}")
print(f"Кількість речень у тестовій вибірці: {len(test_sents)}")

# Створення словника (Vocabulary)
vocab = {}
vocab["--n--"] = 0 # Спеціальний токен кінця речення
vocab["--unk--"] = 1 # Базовий невідомий
# Додаємо спецтокени з assign_unk_pt
special_tokens = ["--unk_digit--", "--unk_punct--", "--unk_upper--",
                  "--unk_noun--", "--unk_verb--", "--unk_adj--", "--unk_adv--"]
for t in special_tokens:
    vocab[t] = len(vocab)

# Заповнюємо словник словами з тренувальної вибірки
# Ми фільтруємо слова, що зустрічаються < 2 разів, щоб модель вчилася працювати з UNK,
# але для спрощення лабораторної додамо всі слова.
for sent in train_sents:
    for word, tag in sent:
        if word not in vocab:
            vocab[word] = len(vocab)

print(f"Розмір словника: {len(vocab)}")

# --- 3. СТВОРЕННЯ МАТРИЦЬ HMM ---

def create_dictionaries(training_corpus, vocab):
    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)

    prev_tag = '--s--' # Start tag

    for sent in training_corpus:
        prev_tag = '--s--'
        for word, tag in sent:
            transition_counts[(prev_tag, tag)] += 1

            # Якщо слово є в словнику, використовуємо його, інакше - UNK логіку
            if word in vocab:
                emission_counts[(tag, word)] += 1
            else:
                unk_token = assign_unk_pt(word)
                emission_counts[(tag, unk_token)] += 1

            tag_counts[tag] += 1
            prev_tag = tag

    return emission_counts, transition_counts, tag_counts

def create_transition_matrix(alpha, tag_counts, transition_counts):
    all_tags = sorted(tag_counts.keys())
    num_tags = len(all_tags)
    A = np.zeros((num_tags, num_tags))

    for i in range(num_tags):
        for j in range(num_tags):
            count = transition_counts.get((all_tags[i], all_tags[j]), 0)
            count_prev = tag_counts[all_tags[i]]
            A[i, j] = (count + alpha) / (count_prev + alpha * num_tags)
    return A, all_tags

def create_emission_matrix(alpha, tag_counts, emission_counts, vocab_list):
    all_tags = sorted(tag_counts.keys())
    num_tags = len(all_tags)
    num_words = len(vocab_list)
    B = np.zeros((num_tags, num_words))

    for i in range(num_tags):
        tag = all_tags[i]
        count_tag = tag_counts[tag]

        # Для оптимізації пройдемося тільки по словах, які цей тег емітував
        # Але для повної матриці треба заповнити все.
        # Це може бути повільно для великого словника.
        # Тому тут зробимо ітерацію по словнику.

        for j, word in enumerate(vocab_list):
            count = emission_counts.get((tag, word), 0)
            B[i, j] = (count + alpha) / (count_tag + alpha * num_words)

    return B

# Генерація матриць
print("Створення матриць частот...")
emission_counts, transition_counts, tag_counts = create_dictionaries(train_sents, vocab)
states = sorted(tag_counts.keys())
print(f"Кількість унікальних тегів: {len(states)}")

print("Створення матриць ймовірностей (A та B)...")
alpha = 0.001
vocab_list = list(vocab.keys())
# Оновлюємо vocab, щоб порядок відповідав vocab_list (для індексації у Viterbi)
vocab_map = {word: i for i, word in enumerate(vocab_list)}

A, _ = create_transition_matrix(alpha, tag_counts, transition_counts)
B = create_emission_matrix(alpha, tag_counts, emission_counts, vocab_list)
print("Матриці готові.")

# --- 4. АЛГОРИТМ ВІТЕРБІ ---

def initialize_viterbi(states, tag_counts, A, B, sent, vocab_map):
    num_tags = len(states)
    best_probs = np.zeros((num_tags, len(sent)))
    best_paths = np.zeros((num_tags, len(sent)), dtype=int)

    # Початковий стан --s--. У нас його немає в матрицях явно як рядка
    # (бо ми брали keys з tag_counts), але ми маємо переходи з '--s--' у transition_counts.
    # Для спрощення припустимо рівномірний старт або використаємо переходи з '--s--'.

    # Обчислимо початкові ймовірності (Start Probabilities)
    # P(tag | start)
    start_probs = np.zeros(num_tags)
    total_starts = sum(transition_counts[('--s--', t)] for t in states)
    for i, tag in enumerate(states):
        count = transition_counts[('--s--', tag)]
        start_probs[i] = (count + alpha) / (total_starts + alpha * num_tags)

    first_word = sent[0][0] # sent is list of (word, tag), need word

    # Обробка першого слова
    if first_word in vocab_map:
        word_idx = vocab_map[first_word]
    else:
        word_idx = vocab_map[assign_unk_pt(first_word)]

    for i in range(num_tags):
        # log(P(start->tag)) + log(P(tag emits word))
        best_probs[i, 0] = math.log(start_probs[i]) + math.log(B[i, word_idx])

    return best_probs, best_paths

def viterbi_forward(A, B, sent, best_probs, best_paths, vocab_map):
    num_tags = best_probs.shape[0]

    for i in range(1, len(sent)):
        word = sent[i][0]
        if word in vocab_map:
            word_idx = vocab_map[word]
        else:
            word_idx = vocab_map[assign_unk_pt(word)]

        for j in range(num_tags): # Поточний тег
            best_prob_i = float("-inf")
            best_path_i = None

            for k in range(num_tags): # Попередній тег
                # prob = prev_prob + trans_prob + emiss_prob
                prob = best_probs[k, i-1] + math.log(A[k, j]) + math.log(B[j, word_idx])

                if prob > best_prob_i:
                    best_prob_i = prob
                    best_path_i = k

            best_probs[j, i] = best_prob_i
            best_paths[j, i] = best_path_i

    return best_probs, best_paths

def viterbi_backward(best_probs, best_paths, states):
    m = best_probs.shape[1] # length of sentence
    z = [None] * m
    pred = [None] * m

    best_prob_last = float('-inf')
    for k in range(len(states)):
        if best_probs[k, m-1] > best_prob_last:
            best_prob_last = best_probs[k, m-1]
            z[m-1] = k

    pred[m-1] = states[z[m-1]]

    for i in range(m-2, -1, -1):
        z[i] = best_paths[z[i+1], i+1]
        pred[i] = states[z[i]]

    return pred

# --- 5. ОЦІНКА ТОЧНОСТІ ---

def evaluate_hmm(test_data, states, tag_counts, A, B, vocab_map):
    correct = 0
    total = 0

    for sent in test_data: # Беремо перші 100 для швидкості, якщо повільно
        if len(sent) == 0: continue

        # Viterbi
        best_probs, best_paths = initialize_viterbi(states, tag_counts, A, B, sent, vocab_map)
        best_probs, best_paths = viterbi_forward(A, B, sent, best_probs, best_paths, vocab_map)
        pred_tags = viterbi_backward(best_probs, best_paths, states)

        # Comparison
        for (word, true_tag), pred_tag in zip(sent, pred_tags):
            if true_tag == pred_tag:
                correct += 1
            total += 1

    return correct / total

print("Оцінка точності HMM на тестовій вибірці...")
# Тестуємо на підмножині для швидкості (або прибрати зріз [:500] для повного тесту)
hmm_accuracy = evaluate_hmm(test_sents[:500], states, tag_counts, A, B, vocab_map)
print(f"Точність HMM (Floresta PT): {hmm_accuracy:.4f}")

# --- 6. ПОРІВНЯННЯ З NLTK ---

print("Навчання NLTK Unigram+Bigram tagger...")
# NLTK потребує дані у форматі list of lists of tuples, що ми вже маємо (train_sents)
t0 = nltk.DefaultTagger('n') # За замовчуванням іменник
t1 = nltk.UnigramTagger(train_sents, backoff=t0)
t2 = nltk.BigramTagger(train_sents, backoff=t1)

nltk_accuracy = t2.evaluate(test_sents[:500])
print(f"Точність NLTK BigramTagger: {nltk_accuracy:.4f}")

print("-" * 30)
print(f"Різниця: {hmm_accuracy - nltk_accuracy:.4f}")

[nltk_data] Downloading package floresta to /root/nltk_data...
[nltk_data]   Unzipping corpora/floresta.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Кількість речень у навчальній вибірці: 7412
Кількість речень у тестовій вибірці: 1854
Розмір словника: 24708
Створення матриць частот...
Кількість унікальних тегів: 37
Створення матриць ймовірностей (A та B)...
Матриці готові.
Оцінка точності HMM на тестовій вибірці...
Точність HMM (Floresta PT): 0.9090
Навчання NLTK Unigram+Bigram tagger...
Точність NLTK BigramTagger: 0.9001
------------------------------
Різниця: 0.0089


/tmp/ipython-input-1407902242.py:297: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  nltk_accuracy = t2.evaluate(test_sents[:500])
